In [1]:
# BOOTSTRAP: luôn tự cấp dữ liệu cho GH Actions
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")  # An toàn trên runner headless
import matplotlib.pyplot as plt

# Đường dẫn
DATA_DIR = Path("../data").resolve()
OUT_SUMMARY_DIR = Path("../result_summary").resolve()
OUT_PLOTS_DIR   = Path("../out/plots").resolve()
OUT_SUMMARY_DIR.mkdir(parents=True, exist_ok=True)
OUT_PLOTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"[i] DATA_DIR    = {DATA_DIR}")
print(f"[i] SUMMARY_DIR = {OUT_SUMMARY_DIR}")
print(f"[i] PLOTS_DIR   = {OUT_PLOTS_DIR}")

def parse_wind_speed(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().replace(",", ".")
    # Bỏ mọi ký tự không phải số/âm/chấm (loại ' km/h')
    s = pd.Series([s]).str.replace(r"[^0-9\.\-]", "", regex=True).iloc[0]
    try: return float(s)
    except: return np.nan

def parse_humidity(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().replace("%","").replace(",", ".")
    try: return float(s)
    except: return np.nan

def map_weather_icon(icon_path: str) -> str:
    if pd.isna(icon_path):
        return "Unknown"
    p = str(icon_path).lower()
    if "rain" in p:     return "Mưa"
    if "cloud" in p:    return "Mây"
    if "sun" in p or "clear" in p: return "Nắng"
    if "fog" in p or "mist" in p:  return "Sương mù"
    if "storm" in p or "thunder" in p: return "Dông bão"
    if "snow" in p:     return "Tuyết"
    return "Khác"

# Load đủ cột để mọi phân tích đều dùng được
def load_all_cities_full(root: Path) -> pd.DataFrame:
    if not root.exists():
        raise FileNotFoundError(f"Không thấy thư mục dữ liệu: {root}")
    dfs = []
    for city_dir in sorted([p for p in root.iterdir() if p.is_dir()]):
        for f in sorted(city_dir.glob("*.csv")):
            try:
                df = pd.read_csv(f)
            except Exception as e:
                print(f"[!] Lỗi đọc {f}: {e}")
                continue

            if "AQI" in df.columns and "aqi" not in df.columns:
                df.rename(columns={"AQI":"aqi"}, inplace=True)

            # Đảm bảo tối thiểu 3 cột này tồn tại
            if "timestamp" in df.columns:
                df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
            else:
                continue
            if "aqi" in df.columns:
                df["aqi"] = pd.to_numeric(df["aqi"], errors="coerce")
            else:
                continue
            if "city" not in df.columns:
                df["city"] = city_dir.name

            keep = [c for c in ["timestamp","city","aqi","wind_speed","humidity","weather_icon"] if c in df.columns]
            df = df[keep].dropna(subset=["timestamp","aqi"])
            if not df.empty:
                dfs.append(df)

    if not dfs:
        raise RuntimeError("Không tìm thấy dữ liệu hợp lệ trong data/*/*.csv")
    out = pd.concat(dfs, ignore_index=True)

    # Số hoá wind/humidity
    out["wind_speed_num"] = out["wind_speed"].apply(parse_wind_speed) if "wind_speed" in out.columns else np.nan
    out["humidity_num"]   = out["humidity"].apply(parse_humidity)     if "humidity"   in out.columns else np.nan

    global HAS_WIND, HAS_HUM, HAS_WEATHER_ICON
    HAS_WIND = pd.notna(out["wind_speed_num"]).any()
    HAS_HUM  = pd.notna(out["humidity_num"]).any()
    HAS_WEATHER_ICON = "weather_icon" in out.columns and out["weather_icon"].notna().any()

    print(f"[i] HAS_WIND={HAS_WIND}, HAS_HUM={HAS_HUM}, HAS_WEATHER_ICON={HAS_WEATHER_ICON}")
    return out

df_full = load_all_cities_full(DATA_DIR)
df_full["month"] = df_full["timestamp"].dt.to_period("M").astype(str)
df_full["hour"]  = df_full["timestamp"].dt.hour

# Lưu raw hợp nhất (tuỳ chọn)
RAW_OUT = OUT_SUMMARY_DIR / "all_aqi_raw_full.csv"
df_full.to_csv(RAW_OUT, index=False, encoding="utf-8-sig")
print(f"[✓] Ghi dữ liệu gộp: {RAW_OUT}")

[i] DATA_DIR    = /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/data
[i] SUMMARY_DIR = /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary
[i] PLOTS_DIR   = /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots


[i] HAS_WIND=True, HAS_HUM=True, HAS_WEATHER_ICON=True


/tmp/ipykernel_2148/386638321.py:96: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_full["month"] = df_full["timestamp"].dt.to_period("M").astype(str)


[✓] Ghi dữ liệu gộp: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/all_aqi_raw_full.csv


# 1. PHÂN TÍCH BIẾN ĐỘNG CHẤT LƯỢNG KHÔNG KHÍ THEO THỜI GIAN

In [2]:
# 1.1. Trung bình AQI theo tháng (so sánh tất cả tỉnh)
monthly = (
    df_full.groupby(["city","month"])["aqi"]
           .agg(mean_aqi="mean", max_aqi="max", min_aqi="min", n_obs="size")
           .reset_index()
           .sort_values(["city","month"])
)
monthly["mean_aqi"] = monthly["mean_aqi"].round(2)
monthly_path = OUT_SUMMARY_DIR / "aqi_monthly_summary.csv"
monthly.to_csv(monthly_path, index=False, encoding="utf-8-sig")
print("[✓] Ghi:", monthly_path)

# Vẽ line gộp mean_aqi
pivot_m = monthly.pivot(index="month", columns="city", values="mean_aqi").sort_index()
plt.figure(figsize=(12,5))
pivot_m.plot(marker="o")
plt.title("Xu hướng AQI trung bình theo tháng (tất cả tỉnh)")
plt.xlabel("Tháng (YYYY-MM)")
plt.ylabel("AQI trung bình")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
out_m = OUT_PLOTS_DIR / "trend_mean_all_cities.png"
plt.savefig(out_m, dpi=150); plt.close()
print("[✓] Vẽ:", out_m)

# 1.2. Trung bình AQI theo giờ (so sánh tất cả tỉnh)
hourly = (
    df_full.groupby(["city","hour"])["aqi"]
           .mean()
           .reset_index()
           .rename(columns={"aqi":"mean_aqi"})
           .sort_values(["city","hour"])
)
hourly["mean_aqi"] = hourly["mean_aqi"].round(2)
hourly_path = OUT_SUMMARY_DIR / "aqi_hourly_summary.csv"
hourly.to_csv(hourly_path, index=False, encoding="utf-8-sig")
print("[✓] Ghi:", hourly_path)

pivot_h = hourly.pivot(index="hour", columns="city", values="mean_aqi").sort_index()
plt.figure(figsize=(12,5))
pivot_h.plot(marker="o")
plt.title("Trung bình AQI theo giờ (tất cả tỉnh)")
plt.xlabel("Giờ (0–23)")
plt.ylabel("AQI trung bình")
plt.xticks(range(0,24))
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
out_h = OUT_PLOTS_DIR / "hourly_mean_all_cities.png"
plt.savefig(out_h, dpi=150); plt.close()
print("[✓] Vẽ:", out_h)

# 1.3. Phân bố mức chất lượng không khí theo tỉnh/thành
def categorize_aqi(aqi):
    if aqi <= 50:   return "Tốt (0-50)"
    if aqi <= 100:  return "Trung bình (51-100)"
    if aqi <= 150:  return "Kém (101-150)"
    if aqi <= 200:  return "Xấu (151-200)"
    if aqi <= 300:  return "Rất xấu (201-300)"
    return "Nguy hại (301+)"

dist_df = df_full.copy()
dist_df["AQI_level"] = pd.to_numeric(dist_df["aqi"], errors="coerce").apply(categorize_aqi)
distribution = (dist_df.groupby(["city","AQI_level"], as_index=False)
                        .size()
                        .rename(columns={"size":"count"}))
distribution["percent"] = (distribution["count"] /
                           distribution.groupby("city")["count"].transform("sum") * 100).round(2)
dist_path = OUT_SUMMARY_DIR / "aqi_distribution_by_city.csv"
distribution.to_csv(dist_path, index=False, encoding="utf-8-sig")
print("[✓] Ghi:", dist_path)

# Vẽ stacked bar %
order_levels = ["Tốt (0-50)","Trung bình (51-100)","Kém (101-150)",
                "Xấu (151-200)","Rất xấu (201-300)","Nguy hại (301+)"]
distribution["AQI_level"] = pd.Categorical(distribution["AQI_level"], order_levels, ordered=True)
pivot_pct = distribution.pivot(index="city", columns="AQI_level", values="percent").fillna(0)

plt.figure(figsize=(12,6))
bottom = np.zeros(len(pivot_pct))
for lvl in order_levels:
    vals = pivot_pct[lvl].values if lvl in pivot_pct.columns else np.zeros(len(pivot_pct))
    plt.bar(pivot_pct.index, vals, bottom=bottom, label=lvl)
    bottom = bottom + vals
plt.title("Phân bố mức chất lượng không khí (AQI) theo tỉnh/thành phố")
plt.ylabel("Tỷ lệ (%)"); plt.xlabel("Tỉnh/Thành phố")
plt.xticks(rotation=30, ha="right")
plt.legend(title="Mức AQI", bbox_to_anchor=(1.02,1), loc="upper left")
plt.tight_layout()
out_dist = OUT_PLOTS_DIR / "distribution_aqi_by_city.png"
plt.savefig(out_dist, dpi=150); plt.close()
print("[✓] Vẽ:", out_dist)

[✓] Ghi: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/aqi_monthly_summary.csv


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/trend_mean_all_cities.png
[✓] Ghi: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/aqi_hourly_summary.csv


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/hourly_mean_all_cities.png
[✓] Ghi: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/aqi_distribution_by_city.csv


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/distribution_aqi_by_city.png


# 2. MỐI QUAN HỆ GIỮA AQI VÀ CÁC YẾU TỐ THỜI TIẾT

In [3]:
from scipy.stats import pearsonr, spearmanr

def safe_corr(x, y, method="pearson"):
    x = np.asarray(x, dtype=float); y = np.asarray(y, dtype=float)
    mask = ~(np.isnan(x) | np.isnan(y))
    x = x[mask]; y = y[mask]
    if len(x) < 3: return np.nan, np.nan
    try:
        return pearsonr(x, y) if method=="pearson" else spearmanr(x, y)
    except Exception:
        return np.nan, np.nan

def corr_by_city(df: pd.DataFrame, feature_col: str, label: str, out_csv_name: str, all_plot_name: str, per_city_prefix: str):
    if feature_col not in df.columns:
        print(f"[!] Thiếu cột {feature_col}, bỏ qua.")
        return
    gdf = df.dropna(subset=["aqi", feature_col]).copy()
    if gdf.empty:
        print(f"[!] Không có dữ liệu hợp lệ cho {label}, bỏ qua.")
        return

    rows = []
    for city, g in gdf.groupby("city"):
        r_p, p_p = safe_corr(g["aqi"], g[feature_col], "pearson")
        r_s, p_s = safe_corr(g["aqi"], g[feature_col], "spearman")
        rows.append({
            "city": city,
            "pearson_r": round(r_p, 4) if pd.notna(r_p) else np.nan,
            "pearson_p": p_p,
            "spearman_rho": round(r_s, 4) if pd.notna(r_s) else np.nan,
            "spearman_p": p_s,
            "n": len(g)
        })
    # Tổng thể
    r_p_all, p_p_all = safe_corr(gdf["aqi"], gdf[feature_col], "pearson")
    r_s_all, p_s_all = safe_corr(gdf["aqi"], gdf[feature_col], "spearman")
    rows.append({
        "city": "__ALL__",
        "pearson_r": round(r_p_all, 4) if pd.notna(r_p_all) else np.nan,
        "pearson_p": p_p_all,
        "spearman_rho": round(r_s_all, 4) if pd.notna(r_s_all) else np.nan,
        "spearman_p": p_s_all,
        "n": len(gdf)
    })
    table = pd.DataFrame(rows).sort_values("city").reset_index(drop=True)
    out_csv = OUT_SUMMARY_DIR / out_csv_name
    table.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print("[✓] Ghi:", out_csv)

    # Scatter + Fit (all)
    plt.figure(figsize=(8,6))
    x = gdf[feature_col].values; y = gdf["aqi"].values
    plt.scatter(x, y, alpha=0.4, s=15)
    plt.title(f"Tương quan AQI vs {label} (tất cả tỉnh)")
    plt.xlabel(label); plt.ylabel("AQI")
    if len(x) >= 2 and np.isfinite(x).any() and np.isfinite(y).any():
        # Đường hồi quy
        mask = ~(np.isnan(x) | np.isnan(y))
        if mask.sum() >= 2:
            a, b = np.polyfit(x[mask], y[mask], deg=1)
            xs = np.linspace(np.nanmin(x), np.nanmax(x), 100)
            ys = a*xs + b
            plt.plot(xs, ys)
    plt.tight_layout()
    out_all = OUT_PLOTS_DIR / all_plot_name
    plt.savefig(out_all, dpi=150); plt.close()
    print("[✓] Vẽ:", out_all)

    # Mỗi tỉnh
    for city, g in gdf.groupby("city"):
        plt.figure(figsize=(7,5))
        plt.scatter(g[feature_col], g["aqi"], alpha=0.5, s=15)
        plt.title(f"AQI vs {label} – {city}")
        plt.xlabel(label); plt.ylabel("AQI")
        if len(g) >= 2:
            xx = g[feature_col].values; yy = g["aqi"].values
            m = ~(np.isnan(xx) | np.isnan(yy))
            if m.sum() >= 2:
                a, b = np.polyfit(xx[m], yy[m], deg=1)
                xs = np.linspace(np.nanmin(xx[m]), np.nanmax(xx[m]), 100)
                ys = a*xs + b
                plt.plot(xs, ys)
        plt.tight_layout()
        safe_city = str(city).replace("/", "_").replace("\\", "_").replace(":", "_")
        out_city = OUT_PLOTS_DIR / f"{per_city_prefix}_{safe_city}.png"
        plt.savefig(out_city, dpi=150); plt.close()
        print("[✓] Vẽ:", out_city)

# Wind
if HAS_WIND:
    corr_by_city(df_full, "wind_speed_num",
                 label="Wind Speed (km/h)",
                 out_csv_name="aqi_vs_windspeed_correlation.csv",
                 all_plot_name="scatter_aqi_vs_windspeed_ALL.png",
                 per_city_prefix="scatter_aqi_vs_windspeed")
else:
    print("[i] Bỏ qua phân tích Wind: dataset không có wind_speed.")

# Humidity
if HAS_HUM:
    corr_by_city(df_full, "humidity_num",
                 label="Humidity (%)",
                 out_csv_name="aqi_vs_humidity_correlation.csv",
                 all_plot_name="scatter_aqi_vs_humidity_ALL.png",
                 per_city_prefix="scatter_aqi_vs_humidity")
else:
    print("[i] Bỏ qua phân tích Humidity: dataset không có humidity.")

[✓] Ghi: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/aqi_vs_windspeed_correlation.csv


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_ALL.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Cần Thơ.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Huế.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Hà Nội.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Hải Phòng.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Hồ Chí Minh.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Nha Trang.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_windspeed_Vinh.png


...

[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_ALL.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Cần Thơ.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Huế.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Hà Nội.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Hải Phòng.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Hồ Chí Minh.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Nha Trang.png
[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Vinh.png


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/scatter_aqi_vs_humidity_Đà Nẵng.png


# 3. AQI TRUNG BÌNH THEO LOẠI THỜI TIẾT VÀ TỈNH/THÀNH PHỐ

In [4]:
if HAS_WEATHER_ICON:
    df_weather = df_full[df_full["weather_icon"].notna()].copy()
    df_weather["weather_type"] = df_weather["weather_icon"].apply(map_weather_icon)

    weather_stats = (
        df_weather.groupby(["city","weather_type"])["aqi"]
                  .mean()
                  .reset_index()
                  .rename(columns={"aqi":"mean_aqi"})
    )
    weather_stats["mean_aqi"] = weather_stats["mean_aqi"].round(2)
    out_csv = OUT_SUMMARY_DIR / "aqi_by_weather_and_city.csv"
    weather_stats.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print("[✓] Ghi:", out_csv)

    # Vẽ group bar: weather_type là trục X, mỗi city là 1 series
    pivot_w = weather_stats.pivot(index="weather_type", columns="city", values="mean_aqi").fillna(0)
    plt.figure(figsize=(14,6))
    pivot_w.plot(kind="bar")
    plt.title("AQI trung bình theo loại thời tiết và tỉnh/thành phố")
    plt.xlabel("Loại thời tiết"); plt.ylabel("AQI trung bình")
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout()
    out_plot = OUT_PLOTS_DIR / "aqi_by_weather_and_city.png"
    plt.savefig(out_plot, dpi=150); plt.close()
    print("[✓] Vẽ:", out_plot)
else:
    print("[i] Bỏ qua phần thời tiết: dataset không có weather_icon.")

[✓] Ghi: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/result_summary/aqi_by_weather_and_city.csv


[✓] Vẽ: /home/runner/work/Automatic-Anomaly-Detection-System/Automatic-Anomaly-Detection-System/out/plots/aqi_by_weather_and_city.png
